# DispNet batch 생성

import (tensorflow, numpy) package <br>
numpy use to make numpy array batch data for training <br>
tensorflow use to make FlowNet structure

In [ ]:
import tensorflow as tf
import numpy as np

dataset : Scene Flow Datasets <br>
model : DispNet structure <br>

In [ ]:
from model import DispNet
from dataset import Scene_Flow_disparity

Download Scene Flow Datasets <br>
Make data_paths list for loading data images

In [ ]:
dataset = Scene_Flow_disparity()

Check data method in mode = left, groundTruth mode = disparity

In [ ]:
print(len(dataset.data_paths))
print()
iteration = 0
for dir_ in dataset.data_paths:
    iteration += 1
    if iteration == 200:
        break
    if dataset.data(dir_) is not None:
        print(dir_)
    else:
        print('no left or png path')

# DispNet Simple training<br>

gpu device setting

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

make DispNet simple structure <br>
set training environment(input shape, learning rate, DispNet structure mode, gpu device set)

In [ ]:
import time
import datetime

start=time.clock()

model = DispNet(
    img_height = 540,
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('simple')

make DispNet check point debug

In [ ]:
import matplotlib.pyplot as plt
import random

model_check = DispNet(
    img_height = 540,
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net_check = model.inference('simple')

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import array_to_img
from PIL import Image

train_left_x = []
train_right_x = []
train_y1 = []
train_y2 = []
train_y3 = []
train_y4 = []
train_y5 = []
train_y6 = []

fit_iteration = 0
iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, pr1 = dataset.data(dir_,
                                                    (model.model_in_height, model.model_in_width),
                                                    (model.model_out_height, model.model_out_width))
        pr = array_to_img(pr1)
        tmp = pr.resize((240, 128))
        pr2 = img_to_array(tmp)
        tmp = pr.resize((120, 64))
        pr3 = img_to_array(tmp)
        tmp = pr.resize((60, 32))
        pr4 = img_to_array(tmp)
        tmp = pr.resize((30, 16))
        pr5 = img_to_array(tmp)
        tmp = pr.resize((15, 8))
        pr6 = img_to_array(tmp)
                
        train_left_x.append(left_image)
        train_right_x.append(right_image)
        train_y1.append(pr1)
        train_y2.append(pr2)
        train_y3.append(pr3)
        train_y4.append(pr4)
        train_y5.append(pr5)
        train_y6.append(pr6)

        iteration += 1

    elif iteration >= 100:
        fit_iteration += 1
        hist = net.fit([np.array(train_left_x), (train_right_x)], 
                       [np.array(train_y1), np.array(train_y2), np.array(train_y3), np.array(train_y4), np.array(train_y5), np.array(train_y6)], 
                       batch_size=8, 
                       epochs=100, 
                       validation_split=0.2)
        if not os.path.exists('./checkpoints/'):
            os.makedirs('./checkpoints/')
        net.save_weights('./checkpoints/dispnetSimple(%dth).hdf5' % fit_iteration)
        
        net_check.load_weights('./checkpoints/dispnetSimple(%dth).hdf5' % fit_iteration)
        for i in range(0, 5):
            n = random.randrange(1,101)
            prediction = net_check.predict([np.array(train_left_x)[n:n+1,:], np.array(train_right_x)[n:n+1,:]], batch_size=1)
            fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(20, 10))
            ax0.imshow(np.array(train_y1)[n,:,:,0])
            ax1.imshow(prediction[0,:,:,0])
            plt.show()        
        
        train_left_x = []
        train_right_x = []
        train_y1 = []
        train_y2 = []
        train_y3 = []
        train_y4 = []
        train_y5 = []
        train_y6 = []
        iteration = 0

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/dispnetSimple_for_depth.hdf5')

# DispNet Correlation training<br>

gpu device setting

In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

make DispNet simple structure <br>
set training environment(input shape, learning rate, DispNet structure mode)

In [ ]:
import time
import datetime

model = DispNet(
    img_height = 540, 
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net = model.inference('correlation')

start=time.clock()

make DispNet check point debug

In [ ]:
import matplotlib.pyplot as plt
import random

model_check = DispNet(
    img_height = 540,
    img_width = 960, 
    img_depth = 3, 
    learning_rate = 0.1**4)

net_check = model.inference('correlation')

## training

make training batch input
- train_left_x_batch numpy array
- train_right_x_batch numpy array
<br>

make training batch output
- train_y_batch numpy array
training
- data split in iteration
- no data augmentation

In [ ]:
from tensorflow.python.keras.preprocessing.image import load_img
from tensorflow.python.keras.preprocessing.image import img_to_array
from tensorflow.python.keras.preprocessing.image import array_to_img
from PIL import Image

train_left_x = []
train_right_x = []
train_y1 = []
train_y2 = []
train_y3 = []
train_y4 = []
train_y5 = []
train_y6 = []

fit_iteration = 0
iteration = 0
for dir_ in dataset.data_paths:
    if dataset.data(dir_) is not None:
        left_image, right_image, pr1 = dataset.data(dir_,
                                                    (model.model_in_height, model.model_in_width),
                                                    (model.model_out_height, model.model_out_width))
        pr = array_to_img(pr1)
        tmp = pr.resize((240, 128))
        pr2 = img_to_array(tmp)
        tmp = pr.resize((120, 64))
        pr3 = img_to_array(tmp)
        tmp = pr.resize((60, 32))
        pr4 = img_to_array(tmp)
        tmp = pr.resize((30, 16))
        pr5 = img_to_array(tmp)
        tmp = pr.resize((15, 8))
        pr6 = img_to_array(tmp)
                
        train_left_x.append(left_image)
        train_right_x.append(right_image)
        train_y1.append(pr1)
        train_y2.append(pr2)
        train_y3.append(pr3)
        train_y4.append(pr4)
        train_y5.append(pr5)
        train_y6.append(pr6)

        iteration += 1

    elif iteration >= 100:
        fit_iteration += 1
        hist = net.fit([np.array(train_left_x), (train_right_x)], 
                       [np.array(train_y1), np.array(train_y2), np.array(train_y3), np.array(train_y4), np.array(train_y5), np.array(train_y6)], 
                       batch_size=8, 
                       epochs=100, 
                       validation_split=0.2)
        if not os.path.exists('./checkpoints/'):
            os.makedirs('./checkpoints/')
        net.save_weights('./checkpoints/dispnetCorr(%dth).hdf5' % fit_iteration)
        
        net_check.load_weights('./checkpoints/dispnetCorr(%dth).hdf5' % fit_iteration)
        for i in range(0, 5):
            n = random.randrange(1,101)
            prediction = net_check.predict([np.array(train_left_x)[n:n+1,:], np.array(train_right_x)[n:n+1,:]], batch_size=1)
            fig, (ax0, ax1) = plt.subplots(ncols=2, figsize=(20, 10))
            ax0.imshow(np.array(train_y1)[n,:,:,0])
            ax1.imshow(prediction[0,:,:,0])
            plt.show()        
        
        train_left_x = []
        train_right_x = []
        train_y1 = []
        train_y2 = []
        train_y3 = []
        train_y4 = []
        train_y5 = []
        train_y6 = []
        iteration = 0

training time check
- day
- hours
- minute
- sec

In [ ]:
runtime_sec = time.clock() - start
m, s = divmod(runtime_sec, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
print('runtime : %d days %d:%02d:%02d' % (d, h, m, s))

save trained model weight

In [ ]:
net.save_weights('./checkpoints/dispnetCorr_for_depth.hdf5')